# Distribution Camparison
* Environment require **Python 3+**

In [48]:
import pandas as pd

In [49]:
df = pd.read_csv('DataBase\CSVData.csv')
df['Total Customer'].fillna(value=df['Total Customer'].mean(),inplace=True)

## Normal Distribution Test

In [50]:
from scipy.stats import normaltest

In [51]:
normaltest(df['Total Customer'])

C:\Miniconda3\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


NormaltestResult(statistic=6.6413399862779485, pvalue=0.03612861771734175)

## Chi-squre Test

In [52]:
from scipy.stats import chi2_contingency

In [53]:
pd.crosstab(df['Status'], df['Target'])

Target,N,Y
Status,,
I,2,2
O,1,2
Q,1,0
R,1,3


In [54]:
chi2 = chi2_contingency(pd.crosstab(df['Status'], df['Target']))

In [55]:
print('The results of the independent Chi-square Test are: \n\tChi-square test value = {:.3f}\n\tp-value = {:.3f}'.format(chi2[0],chi2[1]))

The results of the independent Chi-square Test are: 
	Chi-square test value = 2.057
	p-value = 0.561


## Student-T Test

In [56]:
from scipy.stats import ttest_ind

In [57]:
Grp_A = df[df['Target']=='Y']
Grp_B = df[df['Target']=='N']

In [58]:
ind_ttest  = ttest_ind(Grp_A['Total Customer'], Grp_B['Total Customer'], equal_var=False)

In [59]:
print('The results of the independent t-test are: \n\tt-value = {:.3f}\n\tp-value = {:.3f}'.format(ind_ttest[0],ind_ttest[1]))

The results of the independent t-test are: 
	t-value = 3.931
	p-value = 0.006


* **CI of difference mean**

In [60]:
from math import sqrt
from scipy.stats import t

In [61]:
def ttest_Welchs(Var=None, equal_var=False):
    ind_ttest  = ttest_ind(Grp_A[Var], Grp_B[Var], equal_var=equal_var)
    N1 = Grp_A.shape[0]
    N2 = Grp_B.shape[0]
    mean1 = Grp_A[Var].mean()
    mean2 = Grp_B[Var].mean()
    diff_mean = mean1 - mean2
    std1 = Grp_A[Var].std()
    std2 = Grp_B[Var].std()
    sem1 = Grp_A[Var].sem()
    sem2 = Grp_B[Var].sem()
    if equal_var==True: 
        dfree = (N1 + N2 - 2)
        Sp = sqrt(((N1 - 1)*(std1)**2 + (N2 - 1)*(std2)**2) / (1.0*dfree)) 
        MoE = t.ppf(0.975, dfree) * Sp * sqrt(1/(1.0*N1) + 1/(1.0*N2))
    elif equal_var==False:        
        dfree = (((std1**2/(1.0*N1))+(std2**2/(1.0*N2)))**2)/((std1**4)/(1.0*(N1-1)*(N1**2))+(std2**4)/((1.0*N2-1)*(N2**2)))
        MoE = t.ppf(0.975, dfree) * sqrt(sem1**2 + sem2**2) # un-equal Variance MoE
    return('Grp_A : mean is {:.4f}, SEM is {:.5f}\nGrp_B : mean is {:.4f}, SEM is {:.5f}\nThe results of the independent t-test are: \n\tt-value = {:.3f}\n\tp-value = {:.4f}\nThe difference between groups is {:.4f} [{:.4f} - {:.4f}] (mean [95% CI])'.format(mean1,sem1,mean2,sem2,ind_ttest[0],ind_ttest[1],diff_mean, diff_mean - MoE, diff_mean + MoE)) 

In [62]:
print(ttest_Welchs(Var='Total Customer', equal_var=False))

Grp_A : mean is 4584714.5865, SEM is 374203.40792
Grp_B : mean is 967576.8333, SEM is 840703.89513
The results of the independent t-test are: 
	t-value = 3.931
	p-value = 0.0057
The difference between groups is 3617137.7532 [1439999.3782 - 5794276.1282] (mean [95% CI])


* **CI of each groups**

In [63]:
Grp_A_CI = t.interval(0.95, N1-1, loc=Grp_A['Total Customer'].mean(), scale=Grp_A['Total Customer'].sem())
Grp_B_CI = t.interval(0.95, N1-1, loc=Grp_B['Total Customer'].mean(), scale=Grp_B['Total Customer'].sem())

In [64]:
print ('The mean of GroupA is: {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'
       .format(Grp_A['Total Customer'].mean(), Grp_A_CI[0], Grp_A_CI[1]))
print ('The mean  of GroupB is: {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'
       .format(Grp_B['Total Customer'].mean(), Grp_B_CI[0], Grp_B_CI[1]))

The mean of GroupA is: 4584714.587 [3699864.133 to 5469565.040] (mean [95% CI])
The mean  of GroupB is: 967576.833 [-1020371.985 to 2955525.652] (mean [95% CI])


## Z Test

* Two probability comparison

In [65]:
from math import sqrt
from scipy.stats import norm

In [66]:
N1 = len(df[df['Target']=='Y'])
P1 = len(df[df['Status']=='I'])/len(df[df['Target']=='Y'])
N2 = len(df[df['Target']=='N'])
P2 = len(df[df['Status']=='I'])/len(df[df['Target']=='N'])
p = (N1*P1+N2*P2)/(N1+N2)
q = 1-p
print(N1,N2)
print(P1,P2)
print(p,q)

8 6
0.5 0.6666666666666666
0.5714285714285714 0.4285714285714286


In [67]:
z = ((P1-P2)-0)/sqrt(p*q/N1+p*q/N2)
pval = 2*(1 - norm.cdf(abs(z)))

In [68]:
print('The results of the tow sample probability z-test are: \n\tz-value = {:.3f}\n\tp-value = {:.3f}'.format(z,pval))

The results of the tow sample probability z-test are: 
	z-value = -0.624
	p-value = 0.533


* **CI of difference probability**
    * Set 0.975 for 95% CI with two tail

In [69]:
diff_prob = P1-P2
MoE = norm.ppf(0.975)*sqrt((p*q/N1)+(p*q/N2))

In [70]:
print ('The difference between groups is {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'.format(diff_prob, diff_prob - MoE, diff_prob + MoE))

The difference between groups is -0.167 [-0.690 to 0.357] (mean [95% CI])


* **CI of each groups**
    * You can set as below to get 95%CI also
```PYTHON
norm.interval(alpha = 0.95,loc = P1, scale = sqrt((P1*(1-P1))/N1)) 
norm.interval(alpha = 0.95,loc = P2, scale = sqrt((P2*(1-P2))/N2)) 
```

In [71]:
Grp_A_MoE = norm.ppf(0.975)*sqrt((P1*(1-P1))/N1)
Grp_B_MoE = norm.ppf(0.975)*sqrt((P2*(1-P2))/N2)

In [72]:
print ('The probability of GroupA is: {:.3f} [{:.3f} to {:.3f}] (prob [95% CI])'
       .format(P1, P1-Grp_A_MoE, P1+Grp_A_MoE))
print ('The probability  of GroupB is: {:.3f} [{:.3f} to {:.3f}] (prob [95% CI])'
       .format(P2, P2-Grp_B_MoE, P2+Grp_B_MoE))

The probability of GroupA is: 0.500 [0.154 to 0.846] (prob [95% CI])
The probability  of GroupB is: 0.667 [0.289 to 1.044] (prob [95% CI])


## Oneway ANOVA Test

In [73]:
from scipy.stats import f_oneway

In [74]:
Grp_1 = df[df['Status']=='I']
Grp_2 = df[df['Status']=='R']
Grp_3 = df[df['Status']=='O']

In [75]:
anova = f_oneway(Grp_1['Total Customer'], Grp_2['Total Customer'], Grp_3['Total Customer'])

In [76]:
print('The results of the independent F-test are: \n\tF-value = {:4.3f}\n\tp-value = {:4.3f}'.format(anova[0],anova[1]))

The results of the independent F-test are: 
	F-value = 0.124
	p-value = 0.885


* **CI of each groups**

In [77]:
Grp_1_CI = t.interval(0.95, N1-1, loc=Grp_1['Total Customer'].mean(), scale=Grp_1['Total Customer'].sem())
Grp_2_CI = t.interval(0.95, N1-1, loc=Grp_2['Total Customer'].mean(), scale=Grp_2['Total Customer'].sem())
Grp_3_CI = t.interval(0.95, N1-1, loc=Grp_3['Total Customer'].mean(), scale=Grp_3['Total Customer'].sem())

In [78]:
print ('The mean of Group1 is: {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'
       .format(Grp_1['Total Customer'].mean(), Grp_1_CI[0], Grp_1_CI[1]))
print ('The mean  of Group2 is: {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'
       .format(Grp_2['Total Customer'].mean(), Grp_2_CI[0], Grp_2_CI[1]))
print ('The mean  of Group3 is: {:.3f} [{:.3f} to {:.3f}] (mean [95% CI])'
       .format(Grp_3['Total Customer'].mean(), Grp_3_CI[0], Grp_3_CI[1]))

The mean of Group1 is: 2708612.250 [-710651.501 to 6127876.001] (mean [95% CI])
The mean  of Group2 is: 3059079.750 [569733.779 to 5548425.721] (mean [95% CI])
The mean  of Group3 is: 3730893.333 [-586416.798 to 8048203.465] (mean [95% CI])
